In [1]:
import json
from json import JSONEncoder
import pandas as pd
import string

In [ ]:
with open("/Users/scarlettattensil/Downloads/my_file.json", "r") as file:
    data = json.load(file)
    
with open("/Users/scarlettattensil/Downloads/my_file.json", "w") as file:
    json.dump(sampleJson, file, indent = 4, sort_keys=True)

In [61]:
with open("/Users/scarlettattensil/Downloads/events.json", "r") as file:
    data = json.load(file)

df = pd.DataFrame(data)
df.head(3)

,id,trailer_id,facility_id,entrance_time,exit_time
0,57185,15114,319,2022-03-26 19:04:54,None
1,57184,15205,319,2022-03-26 13:54:26,None
2,57165,15172,846,2022-03-26 11:28:01,None


In [150]:
df['entrance'] = pd.to_datetime(df['entrance_time'])
df['exits'] = pd.to_datetime(df['exit_time'])
df

,id,trailer_id,facility_id,entrance_time,exit_time,entrance,exits
0,57185,15114,319,2022-03-26 19:04:54,None,2022-03-26 19:04:54,NaT
1,57184,15205,319,2022-03-26 13:54:26,None,2022-03-26 13:54:26,NaT
2,57165,15172,846,2022-03-26 11:28:01,None,2022-03-26 11:28:01,NaT
3,57164,15171,1,2022-03-25 07:00:00,None,2022-03-25 07:00:00,NaT
4,57163,15170,1,2022-03-25 07:00:00,None,2022-03-25 07:00:00,NaT
...,...,...,...,...,...,...,...
994,56169,14419,846,2022-01-07 08:00:00,2022-01-13 14:08:04,2022-01-07 08:00:00,2022-01-13 14:08:04
995,56168,14412,846,2022-01-07 08:00:00,2022-01-07 20:06:33,2022-01-07 08:00:00,2022-01-07 20:06:33
996,56167,13691,846,2022-01-07 08:00:00,2022-01-07 20:06:33,2022-01-07 08:00:00,2022-01-07 20:06:33
997,56166,14329,846,2022-01-06 20:08:06,2022-03-11 20:08:03,2022-01-06 20:08:06,2022-03-11 20:08:03


In [52]:
def filter_df(df, given_date,facility):
    given = datetime.datetime.strptime(given_date, "%Y-%m-%d").date()
    results = []
    for index,row in df.iterrows():
        enter = datetime.datetime.strptime(row['entrance_time'], "%Y-%m-%d %H:%M:%S").date()
        if row['exit_time']:
            exit = datetime.datetime.strptime(row['exit_time'], "%Y-%m-%d %H:%M:%S").date()
            if (enter <= given) and (exit >= given) and (row['facility_id'] == facility):
                results.append(row)
        else:
            if (enter <= given) and (row['facility_id'] == facility):
                results.append(row)

    return len(results)

In [53]:
filter_df(df, '2022-01-18', 1)

18

In [ ]:
import datetime

In [99]:
def occupancy(df, given_date, facility):
    given_date = datetime.datetime.strptime(given_date, "%Y-%m-%d").date()
    max = 0
    count = 0
    events = []
    for index, row in df.iterrows():
        enter_time = datetime.datetime.strptime(row['entrance_time'], "%Y-%m-%d %H:%M:%S")
        if row['facility_id'] == facility and (enter_time <= given_date):
            events.append(['entry',enter_time])
            if row['exit_time']:
                exit_time = datetime.datetime.strptime(row['exit_time'], "%Y-%m-%d %H:%M:%S")
                events.append(['exit',exit_time])
    
    events_df = pd.DataFrame(events, columns = ['type','time'])
    events_df.sort_values(by = 'time')

    for index, row in events_df.iterrows():
        print(row)
        if row['type'] == 'entry':
            count += 1
            if count > max:
                max = count
        else:
            count -= 1
    
    return max




,id,trailer_id,facility_id,entrance_time,exit_time,entrance,exits


In [170]:
import pandas as pd
import datetime

def occupancy(df, given_date, facility):
    given_date = datetime.datetime.strptime(given_date, "%Y-%m-%d").date()
    
    max_count = 0  
    count = 0  
    events = []  
    
    for index, row in df.iterrows():
        entrance_time = datetime.datetime.strptime(row['entrance_time'], "%Y-%m-%d %H:%M:%S")
        
        # entering before date and leaving after or not at all
        if row['facility_id'] == facility and entrance_time.date() < given_date:
            if not row['exit_time'] or datetime.datetime.strptime(row['exit_time'], "%Y-%m-%d %H:%M:%S").date() >= given_date:
                count += 1

        # entering on the date
        if row['facility_id'] == facility and entrance_time.date() == given_date:
            events.append(['entry', entrance_time, row['trailer_id']])  

        # exiting on the date
        if row['facility_id'] == facility and row['exit_time']:
            exit_time = datetime.datetime.strptime(row['exit_time'], "%Y-%m-%d %H:%M:%S")
            if exit_time.date() == given_date:  
                events.append(['exit', exit_time, row['trailer_id']])

    print('Trailers there at begining of day:', count)
    
    events_df = pd.DataFrame(events, columns=['type', 'time', 'trailer_id'])
    events_df.sort_values(by='time', inplace=True)

    start_time = None
    end_time = None
    
    for index, row in events_df.iterrows():
        if row['type'] == 'entry':
            count += 1
            if count > max_count:
                max_count = count
                start_time = row['time']  
                end_time = row['time']  
        elif row['type'] == 'exit':
            count -= 1
            if count == max_count - 1:  
                end_time = row['time']
    
    return max_count, start_time, end_time


In [161]:
df[df['trailer_id'] == 14502]

,id,trailer_id,facility_id,entrance_time,exit_time,entrance,exits
882,56281,14502,1,2022-01-18 10:10:18,2022-01-19 10:10:16,2022-01-18 10:10:18,2022-01-19 10:10:16
935,56228,14502,1,2022-01-13 08:00:00,2022-01-18 10:10:17,2022-01-13 08:00:00,2022-01-18 10:10:17


In [171]:
occupancy(df, '2022-01-18', 1)

Trailers there at begining of day: 14


(16, Timestamp('2022-01-18 08:00:00'), Timestamp('2022-01-18 10:10:17'))